### I221994 | Muhammad Qasim
### I222066 | Ayaan Khan
### I222018 | Ahmad Luqman

## Import Required Libraries

In [1]:
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms

from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Libraries imported successfully!
PyTorch version: 2.7.0+cu126
CUDA available: True


In [2]:
def auror_defense(grad_vectors, attention_heads=4):
    """
    AUROR: Multi-head attention mechanism to identify malicious updates
    Uses self-attention to weigh client contributions
    
    Args:
        grad_vectors: List of gradient vectors from clients
        attention_heads: Number of attention heads
    
    Returns:
        benign_mask: Boolean array indicating benign clients
    """
    n_clients = len(grad_vectors)
    X = np.array(grad_vectors)
    
    # Simplified multi-head attention
    head_scores = []
    
    for h in range(attention_heads):
        # Random projection for each head
        np.random.seed(42 + h)
        proj_dim = min(100, X.shape[1] // 2)
        W = np.random.randn(X.shape[1], proj_dim) * 0.01
        
        # Project gradients
        X_proj = X @ W
        
        # Compute attention scores (similarity to mean)
        mean_proj = np.mean(X_proj, axis=0)
        scores = np.zeros(n_clients)
        
        for i in range(n_clients):
            # Cosine similarity to mean
            scores[i] = np.dot(X_proj[i], mean_proj) / (
                np.linalg.norm(X_proj[i]) * np.linalg.norm(mean_proj) + 1e-8
            )
        
        head_scores.append(scores)
    
    # Average scores across heads
    avg_scores = np.mean(head_scores, axis=0)
    
    # Select top clients (above median)
    threshold = np.median(avg_scores)
    benign_mask = avg_scores >= threshold
    
    return benign_mask

In [3]:
def rfa_defense(grad_vectors, max_iter=100, tol=1e-5):
    """
    RFA: Uses geometric median instead of mean (more robust to outliers)
    Weierstrass algorithm for geometric median computation
    
    Args:
        grad_vectors: List of gradient vectors from clients
        max_iter: Maximum iterations for geometric median
        tol: Convergence tolerance
    
    Returns:
        benign_mask: Boolean array indicating benign clients
    """
    n_clients = len(grad_vectors)
    X = np.array(grad_vectors)
    
    # Compute geometric median using Weierstrass algorithm
    median = np.median(X, axis=0)  # Initialize with coordinate median
    
    for iteration in range(max_iter):
        prev_median = median.copy()
        
        # Compute distances to current median
        distances = np.linalg.norm(X - median, axis=1)
        
        # Avoid division by zero
        distances = np.maximum(distances, 1e-8)
        
        # Weighted update
        weights = 1.0 / distances
        weights = weights / np.sum(weights)
        
        median = np.sum(X * weights[:, np.newaxis], axis=0)
        
        # Check convergence
        if np.linalg.norm(median - prev_median) < tol:
            break
    
    # Compute distances to geometric median
    distances_to_median = np.linalg.norm(X - median, axis=1)
    
    # Use MAD (Median Absolute Deviation) for robust thresholding
    mad = np.median(np.abs(distances_to_median - np.median(distances_to_median)))
    threshold = np.median(distances_to_median) + 2.5 * mad
    
    benign_mask = distances_to_median <= threshold
    
    return benign_mask

## Model Definitions

In [4]:
class SimpleCNN(nn.Module):
    """Simple CNN for MNIST and Fashion-MNIST"""
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7 if input_channels==1 else 64*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        return self.fc(self.conv(x))

def get_resnet18(num_classes=10):
    """ResNet18 adapted for CIFAR-10"""
    model = torchvision.models.resnet18(pretrained=False)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

print("Model architectures defined!")

Model architectures defined!


## Utility Functions

In [5]:
def model_to_vector(model):
    """Convert model parameters to a flat numpy vector"""
    return torch.cat([p.detach().flatten() for p in model.parameters()]).cpu().numpy()

def vector_to_model(model, vec):
    """Load vector into model parameters"""
    ptr = 0
    for p in model.parameters():
        num = p.numel()
        p.data.copy_(torch.tensor(vec[ptr:ptr+num]).view_as(p.data))
        ptr += num

def subtract_model_params(m_new, m_old):
    """Compute parameter update: new - old"""
    v_new = model_to_vector(m_new)
    v_old = model_to_vector(m_old)
    return v_new - v_old

def set_seed(seed):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

print("Utility functions defined!")

Utility functions defined!


## Attack Implementations

In [6]:
def sign_flip_update(grad_vec):
    """Sign flipping attack: negate the gradient update"""
    return -grad_vec

def lie_attack_simulation(global_mean, global_std, beta=1.5):
    """LIE attack: craft malicious update near mean but in negative direction"""
    sign = -1.0
    return global_mean + sign * beta * global_std

print("Attack functions defined!")

Attack functions defined!


## Federated Learning Utilities

In [7]:
def local_train(model, train_loader, device, epochs=1, lr=0.01):
    """Train model locally for specified epochs"""
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for e in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

def fedavg(models):
    """Federated Averaging: average model parameters"""
    avg = {}
    for k in models[0].keys():
        avg[k] = sum([m[k].float() for m in models]) / len(models)
    return avg

def evaluate(model, dataloader, device):
    """Evaluate model accuracy on test set"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    
    return correct / total

print("Federated learning functions defined!")

Federated learning functions defined!


## NC-FLD Defense: Weight Pruning, PCA, and Classification

In [8]:
def select_top_by_magnitude(grad_vec, keep_ratio):
    """Select top parameters by magnitude (weight pruning step)"""
    magnitudes = np.abs(grad_vec)
    threshold = np.quantile(magnitudes, 1 - keep_ratio)
    mask = magnitudes > threshold
    return grad_vec[mask], mask

def build_feature_matrix(selected_vectors):
    """Build feature matrix from selected parameters (pad to max length)"""
    max_len = max([v.shape[0] for v in selected_vectors])
    X = np.zeros((len(selected_vectors), max_len), dtype=np.float32)
    for i, v in enumerate(selected_vectors):
        X[i, :v.shape[0]] = v
    return X

def apply_pca(X, n_components=2):
    """Apply PCA for dimensionality reduction"""
    pca = PCA(n_components=n_components)
    Z = pca.fit_transform(X)
    return Z, pca

def classify_embeddings(Z, method='ocsvm'):
    """Classify clients as benign (True) or malicious (False)"""
    if method == 'ocsvm':
        clf = OneClassSVM(kernel='rbf', nu=0.1, gamma='scale')
        clf.fit(Z)
        preds = clf.predict(Z)  # +1 inlier, -1 outlier
        return (preds == 1).astype(bool)
    
    elif method == 'agg':
        cl = AgglomerativeClustering(n_clusters=2, linkage='average')
        labels = cl.fit_predict(Z)
        counts = np.bincount(labels)
        benign_label = np.argmax(counts)
        return (labels == benign_label).astype(bool)
    
    elif method == 'knn':
        k = 4
        kn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
        agg = AgglomerativeClustering(n_clusters=2)
        pseudo = agg.fit_predict(Z)
        counts = np.bincount(pseudo)
        benign_label = np.argmax(counts)
        y = (pseudo == benign_label).astype(int)
        kn.fit(Z, y)
        preds = kn.predict(Z)
        return (preds == 1).astype(bool)
    elif method == 'auror':
        aur = auror_defense(Z, attention_heads=4)
        return aur
    elif method == 'rfa':
        rffaa = rfa_defense(Z, max_iter=100)
        return rffaa
        
    
    else:
        raise ValueError(f'Unknown defense method: {method}')

print("NC-FLD defense functions defined!")

NC-FLD defense functions defined!


## Main Simulation Function

In [9]:
# Create named tuple for experiment configuration
ExpConfig = namedtuple('ExpConfig', ['dataset', 'clients', 'rounds', 'local_epochs', 
                                     'lr', 'attack', 'malicious_ratio', 'defense', 
                                     'keep_ratio', 'seed', 'log_interval'])

def simulate(config):
    """
    Run federated learning simulation with NC-FLD defense
    
    Args:
        config: ExpConfig object with experiment parameters
    
    Returns:
        results: List of test accuracies per round
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    set_seed(config.seed)
    
    # Dataset setup
    if config.dataset == 'mnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                             download=True, transform=transform)
        testset = torchvision.datasets.MNIST(root='./data', train=False, 
                                            download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 32
        flip_map = {7: 1}
    
    elif config.dataset == 'fmnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, 
                                                     download=True, transform=transform)
        testset = torchvision.datasets.FashionMNIST(root='./data', train=False, 
                                                    download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 25
        flip_map = {0: 6, 6: 0}
    
    elif config.dataset == 'cifar10':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5, 0.5, 0.5), 
                                                          (0.5, 0.5, 0.5))])
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                               download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                              download=True, transform=transform)
        model_fn = lambda: get_resnet18(num_classes=10)
        batch_size = 100
        flip_map = {5: 3}  # dog -> cat
    
    else:
        raise NotImplementedError(f'Dataset {config.dataset} not implemented')
    
    # Create client data shards
    total_train = len(trainset)
    indices = list(range(total_train))
    random.shuffle(indices)
    shards = np.array_split(indices, config.clients)
    
    client_loaders = []
    for s in shards:
        sub = Subset(trainset, s)
        loader = DataLoader(sub, batch_size=batch_size, shuffle=True)
        client_loaders.append(loader)
    
    testloader = DataLoader(testset, batch_size=256, shuffle=False)
    
    # Select malicious clients
    num_mal = int(config.malicious_ratio * config.clients)
    mal_indices = set(random.sample(range(config.clients), num_mal))
    print(f"Malicious clients: {sorted(list(mal_indices))} (count={num_mal})")
    
    # Initialize global model
    global_model = model_fn().to(device)
    global_state = copy.deepcopy(global_model.state_dict())
    
    # Main federated learning rounds
    results = []
    for r in trange(config.rounds, desc='Training Rounds'):
        local_states = []
        grad_vectors = []
        raw_masks = []
        
        # Local training for each client
        for c in range(config.clients):
            model_c = model_fn().to(device)
            model_c.load_state_dict(global_state)
            old_model = copy.deepcopy(model_c)
            
            # Label flip attack: poison training data
            if config.attack == 'label_flip' and c in mal_indices:
                ds_indices = shards[c]
                poison_list = []
                for idx in ds_indices:
                    x, y = trainset[idx]
                    y_ = flip_map.get(int(y), int(y))
                    poison_list.append((x, y_))
                poisoned_loader = DataLoader(poison_list, batch_size=batch_size, shuffle=True)
                local_train(model_c, poisoned_loader, device, 
                          epochs=config.local_epochs, lr=config.lr)
            else:
                local_train(model_c, client_loaders[c], device, 
                          epochs=config.local_epochs, lr=config.lr)
            
            # Compute update vector
            diff_vec = subtract_model_params(model_c, old_model)
            
            # Model poisoning attacks
            if c in mal_indices and config.attack == 'sign_flip':
                diff_vec = sign_flip_update(diff_vec)
            
            if c in mal_indices and config.attack == 'lie':
                if len(grad_vectors) > 0:
                    global_mean = np.mean(np.stack(grad_vectors), axis=0)
                    global_std = np.std(np.stack(grad_vectors), axis=0) + 1e-6
                    diff_vec = lie_attack_simulation(global_mean, global_std, beta=1.0)
                else:
                    diff_vec = sign_flip_update(diff_vec)
            
            grad_vectors.append(diff_vec)
            local_states.append(model_c.state_dict())
        
        # NC-FLD Defense: Weight Pruning
        selected_vecs = []
        for vec in grad_vectors:
            sel, mask = select_top_by_magnitude(vec, config.keep_ratio)
            selected_vecs.append(sel)
            raw_masks.append(mask)
        
        # Build feature matrix and apply PCA
        X = build_feature_matrix(selected_vecs)
        Z, pca = apply_pca(X, n_components=2)
        
        # Classify clients
        benign_mask = classify_embeddings(Z, method=config.defense)
        benign_indices = [i for i, b in enumerate(benign_mask) if b]
        
        if len(benign_indices) == 0:
            print("Warning: all clients classified malicious; skipping aggregation")
            continue
        
        # Aggregate benign clients (FedAvg)
        benign_states = [local_states[i] for i in benign_indices]
        new_global_state = fedavg(benign_states)
        global_state = new_global_state
        
        # Evaluate
        global_model.load_state_dict(global_state)
        acc = evaluate(global_model, testloader, device)
        results.append(acc)
        
        if (r + 1) % config.log_interval == 0:
            print(f"Round {r+1}/{config.rounds} - Test Acc: {acc:.4f} - "
                  f"Included: {len(benign_indices)}/{config.clients}")
    
    final_acc = results[-1] if len(results) > 0 else None
    print(f"Final test accuracy: {final_acc:.4f}")
    
    # Save the final model
    import os
    os.makedirs('models', exist_ok=True)
    model_filename = f"models/{config.dataset}_{config.attack}_mr{config.malicious_ratio}_{config.defense}_final.pth"
    torch.save(global_model.state_dict(), model_filename)
    print(f"Model saved to: {model_filename}")
    
    return results, final_acc

print("Simulation function defined!")

Simulation function defined!


## Initialize Results Storage

In [10]:
# Storage for all experiment results
all_results = []

print("Results storage initialized!")

Results storage initialized!


---
# MNIST Experiments (20 clients)
## Label Flip Attack

In [11]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=20, rounds=50, local_epochs=1, lr=0.01,
                       attack='label_flip', malicious_ratio=ratio, defense='auror',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': ratio,
                       'defense': 'auror', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

100%|██████████| 9.91M/9.91M [00:03<00:00, 2.50MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 128kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


Malicious clients: [2, 13] (count=2)


Training Rounds:  20%|██        | 10/50 [02:19<09:19, 13.98s/it]

Round 10/50 - Test Acc: 0.9262 - Included: 10/20


Training Rounds:  40%|████      | 20/50 [04:33<06:42, 13.41s/it]

...

Round 30/50 - Test Acc: 0.8724 - Included: 10/20


Training Rounds:  80%|████████  | 40/50 [09:57<02:31, 15.15s/it]

Round 40/50 - Test Acc: 0.8766 - Included: 10/20


Training Rounds: 100%|██████████| 50/50 [12:30<00:00, 15.02s/it]

Round 50/50 - Test Acc: 0.8802 - Included: 10/20
Final test accuracy: 0.8802
Model saved to: models/mnist_label_flip_mr0.6_auror_final.pth


In [12]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=20, rounds=50, local_epochs=1, lr=0.01,
                       attack='sign_flip', malicious_ratio=ratio, defense='auror',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': ratio,
                       'defense': 'auror', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [2, 13] (count=2)


Training Rounds:  20%|██        | 10/50 [02:19<09:07, 13.69s/it]

Round 10/50 - Test Acc: 0.9285 - Included: 10/20


Training Rounds:  40%|████      | 20/50 [04:36<06:52, 13.76s/it]

Round 20/50 - Test Acc: 0.9580 - Included: 10/20


...

Round 30/50 - Test Acc: 0.9702 - Included: 10/20


Training Rounds:  80%|████████  | 40/50 [08:36<02:10, 13.02s/it]

Round 40/50 - Test Acc: 0.9759 - Included: 10/20


Training Rounds: 100%|██████████| 50/50 [10:46<00:00, 12.92s/it]

Round 50/50 - Test Acc: 0.9785 - Included: 10/20
Final test accuracy: 0.9785
Model saved to: models/mnist_sign_flip_mr0.6_auror_final.pth


In [13]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=20, rounds=50, local_epochs=1, lr=0.01,
                       attack='lie', malicious_ratio=ratio, defense='auror',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': ratio,
                       'defense': 'auror', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [2, 13] (count=2)


Training Rounds:  20%|██        | 10/50 [02:09<08:37, 12.94s/it]

Round 10/50 - Test Acc: 0.9290 - Included: 10/20


Training Rounds:  40%|████      | 20/50 [04:19<06:27, 12.91s/it]

Round 20/50 - Test Acc: 0.9582 - Included: 10/20


...

Round 30/50 - Test Acc: 0.9689 - Included: 10/20


Training Rounds:  80%|████████  | 40/50 [08:59<02:11, 13.17s/it]

Round 40/50 - Test Acc: 0.9757 - Included: 10/20


Training Rounds: 100%|██████████| 50/50 [11:10<00:00, 13.40s/it]

Round 50/50 - Test Acc: 0.9793 - Included: 10/20
Final test accuracy: 0.9793
Model saved to: models/mnist_lie_mr0.6_auror_final.pth


In [14]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='lie', malicious_ratio=ratio, defense='rfa',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': ratio,
                       'defense': 'rfa', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [5, 8, 20, 41, 54, 59, 72, 80, 83, 84] (count=10)


Training Rounds:  10%|█         | 10/100 [02:24<21:33, 14.38s/it]

Round 10/100 - Test Acc: 0.6858 - Included: 89/100


Training Rounds:  20%|██        | 20/100 [04:49<19:21, 14.51s/it]

Round 20/100 - Test Acc: 0.8705 - Included: 83/100


...

Round 80/100 - Test Acc: 0.9506 - Included: 100/100


Training Rounds:  90%|█████████ | 90/100 [26:21<02:55, 17.57s/it]

Round 90/100 - Test Acc: 0.9562 - Included: 100/100


Training Rounds: 100%|██████████| 100/100 [29:18<00:00, 17.58s/it]

Round 100/100 - Test Acc: 0.9599 - Included: 100/100
Final test accuracy: 0.9599
Model saved to: models/mnist_lie_mr0.6_rfa_final.pth
